# Imports

In [ ]:
from pathlib import Path
from asapdiscovery.data.openeye import load_openeye_sdfs, oechem, save_openeye_sdfs
from asapdiscovery.data.fragalysis import parse_fragalysis
import numpy as np
import pandas as pd, numpy as np
import plotly.express as px
from datetime import datetime
from tqdm.notebook import tqdm
from asapdiscovery.docking.analysis import DockingResults
from importlib import reload

# Load Paths

In [ ]:
import sys
sys.path.append(str(Path("../../../").resolve()))
import software.paths as p

In [ ]:
reload(p)

In [ ]:
paths = p.paths

In [ ]:
paths.sars_hybrid.exists()

## loadall csvs

In [ ]:
csvs = paths.sars_hybrid.glob("run_docking_oe.*-results.csv")

In [ ]:
print("start", datetime.now())
df = pd.concat(map(pd.read_csv, csvs), ignore_index= True)
print("end", datetime.now())

In [ ]:
len(df)

## save initial combined csv

In [ ]:
combined_csv_path = paths.sars_hybrid / "run_docking_oe-results.csv"

In [ ]:
df.to_csv(combined_csv_path, index=False)

## Load csv

In [ ]:
dr = DockingResults(csv_path=str(combined_csv_path), column_names="None")

In [ ]:
len(dr.df)

## only 17435 results were collected

## Clean Up CSV

In [ ]:
dr.df.columns

In [ ]:
dr.df.columns = ["Compound_ID", 
                 "Structure_Source", 
                 "Docked_File", 
                 "Pose_ID", 
                 "RMSD", 
                 "POSIT", 
                 "POSIT_Method", 
                 "Chemgauss4", 
                 "Clash", 
                 "SMILES", 
                 "GAT_Score",
                "SCHNET_score"]

In [ ]:
dr.df["POSIT_R"] = 1-dr.df["POSIT"]
dr.df["Complex_ID"] = dr.df.Compound_ID.apply(str) + "_" + dr.df.Structure_Source

In [ ]:
dr.df

In [ ]:
dr.df.POSIT_Method.unique()

# Add Compound Info

In [ ]:
from asapdiscovery.data.utils import get_compound_id_xtal_dicts

In [ ]:
with open("/Users/alexpayne/Scientific_Projects/covid-moonshot-ml/metadata/cmpd_to_frag.yaml") as f:
    cmpd_to_frag_dict = yaml.safe_load(f)

In [ ]:
dataset_df = pd.DataFrame({"Compound_ID": cmpd_to_frag_dict.keys(), 
                           "Compound_Source": [source for source in cmpd_to_frag_dict.values()]})

In [ ]:
dataset_df

In [ ]:
dr.df = pd.merge(dr.df, dataset_df, on="Compound_ID")

In [ ]:
dr.df.head()

## Add Reference Compound

In [ ]:
frag_to_cmpd_dict = {v:k for k,v in cmpd_to_frag_dict.items()}

In [ ]:
reference_df = pd.DataFrame({"Compound_ID": cmpd_to_frag_dict.keys(), 
                           "Structure_Source": [source for source in cmpd_to_frag_dict.values()]})

In [ ]:
Structure_Source = dr.df.Structure_Source.apply(lambda x: x.split("_")[0])

In [ ]:
ref_lig = Structure_Source.apply(lambda x: frag_to_cmpd_dict[x])

In [ ]:
dr.df["Reference_Ligand"] = ref_lig
dr.df["Structure_Name"] = Structure_Source

# Add Dates

In [ ]:
mpro_soaks = paths.fragalysis / "extra_files/Mpro_soaks.csv"

In [ ]:
mpro_soaks.exists()

In [ ]:
date_df = pd.read_csv(mpro_soaks)

In [ ]:
ddf = date_df.loc[:, ["Sample Name", "Data Collection Date"]]

In [ ]:
ddf.head()

In [ ]:
def date_processor(date_string):
    if type(date_string) == str and not date_string == 'None':
        try:
            return datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S").date()
        except ValueError:
            return datetime.strptime(date_string, "%d/%m/%Y %H:%M").date()
    else:
        return None

In [ ]:
to_merge = pd.DataFrame({"Structure_Name":ddf["Sample Name"], "Structure_Date":ddf["Data Collection Date"].apply(date_processor)})

In [ ]:
merged = pd.merge(dr.df, to_merge, on="Structure_Name")

In [ ]:
merged.to_csv(combined_csv_path.parent / "results_cleaned.csv")